# Data Clean Up

### Load all the libraries we will need:
- Pandas, for CSV data manipulation
- Seaborn, for pretty graphs
- Matplotlib.pyplot, for less pretty graphs
- Numpy, for some arithmetics

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

### Load files into the notebook

Given the sheer quantity of the files, in the future this should be made into a database to save some lines of code.

In [24]:
cir_df = pd.read_csv('f1_dataset/circuits.csv')  #0
con_res_df = pd.read_csv('f1_dataset/constructor_results.csv') #1
con_sta_df = pd.read_csv('f1_dataset/constructor_standings.csv') #2
con_df = pd.read_csv('f1_dataset/constructors.csv') #3
drv_sta_df = pd.read_csv('f1_dataset/driver_standings.csv') #4
drv_df = pd.read_csv('f1_dataset/drivers.csv') #5
lap_df = pd.read_csv('f1_dataset/lap_times.csv') #6
pit_df = pd.read_csv('f1_dataset/pit_stops.csv') #7
q_df = pd.read_csv('f1_dataset/qualifying.csv') #8
rac_df = pd.read_csv('f1_dataset/races.csv') #9
res_df = pd.read_csv('f1_dataset/results.csv') #10
ssn_df = pd.read_csv('f1_dataset/seasons.csv') #11
spr_res_df = pd.read_csv('f1_dataset/sprint_results.csv') #12
sta_df = pd.read_csv('f1_dataset/status.csv') #13

all_files = [cir_df, con_res_df, con_sta_df, con_df, drv_sta_df, drv_df, lap_df, pit_df, q_df, rac_df, res_df, ssn_df, spr_res_df, sta_df]



### Look for missing data

Only print the dataframes with empty cells


In [31]:
# Print the count of NaNs per column. This acheives the same as DataFrame.info(), but is in a more manageble format
for i, df in enumerate(all_files):
    temp_df = np.sum(df.isna(),axis=0)
    if np.sum(temp_df) > 0:
        print('Dataframe ' + str(i) + ' contains empty fields')
        print(temp_df)

Dataframe 8 contains empty fields
qualifyId          0
raceId             0
driverId           0
constructorId      0
number             0
position           0
q1                 8
q2               164
q3               327
dtype: int64


### Quali dataset
Only ```q_df``` dataframe has empty values. One may have a good idea why that is, looking where the data is missing. Some datapoints are missing from Q1, more from Q2 and most from Q3. This pattern clearly coincides with qualification elimination.

Seeing the data below, looks like the drivers who did not have a Q1 entry still participated in the races. Checking some of the reasons for these occurences we can learnt hat for example in Azerbaijan 2019 GP (raceId 1013), Pierre Gasly and Kimi Raikkonen were disqualified from quali, thus removing their set times. Yet, they were still allowed to participate in the race.

In [47]:
# select just Q1 NaNs
q1_na = q_df.loc[q_df['q1'].isna()]
# understand more about those drivers who didnt have a Q1 timed entry by looking at their race result
q1_more_info = pd.merge(q1_na, res_df,  how='left', left_on=['raceId','driverId'], right_on = ['raceId','driverId'])
# show 
q1_more_info[['raceId','driverId','q1','grid','position_x','position_y']].head(10)

,raceId,driverId,q1,grid,position_x,position_y
0,983,20,NaN,20,20,4
1,993,843,NaN,20,20,12
2,994,830,NaN,20,20,9
3,995,154,NaN,20,20,12
4,1013,8,NaN,0,19,10
5,1013,842,NaN,0,20,\N
6,1020,20,NaN,20,20,2
7,1024,817,NaN,20,20,14


There are too many Q2 and Q3 NaN entries to check them one by one for a specific reason. We can fairly safely judge that most of them are correct due to Q1 and Q2 elimination respectively. 

We can just sense check that all Q2 participants have a Q1 time recorded, and that all Q3 participants have a Q2 time recorded. And indeed - analysis below shows all participants have all the relevant quali times recorded. We therefore can conclude this dataset is complete.

In [53]:
# find every racer who is in Q2
q2_entrants = q_df.loc[~q_df['q2'].isna()]
# see if any of them have a NaN in Q1 entry
q2_entrants.loc[q2_entrants['q1'].isna()]

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3


In [54]:
# find every racer who is in Q2
q3_entrants = q_df.loc[~q_df['q3'].isna()]
# see if any of them have a NaN in Q1 entry
q3_entrants.loc[q3_entrants['q2'].isna()]

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
